In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# additional imports 
import matplotlib.pyplot as plt

# Image metadata

In [ ]:
df_csv = pd.read_csv('../input/airbus-ship-detection/train_ship_segmentations_v2.csv')
df_csv.head()

In [ ]:
df_csv['has_vessel'] = df_csv['EncodedPixels'].notnull()
df_csv['has_vessel'].head()

In [ ]:
def rle_to_pixels(rle_code):
    '''
    Transforms a RLE code string into a list of pixels of a (768, 768) canvas
    
    Source: https://www.kaggle.com/julian3833/2-understanding-and-plotting-rle-bounding-boxes
    '''
    rle_code = [int(i) for i in rle_code.split()]
    pixels = [(pixel_position % 768, pixel_position // 768) 
                 for start, length in list(zip(rle_code[0:-1:2], rle_code[1::2])) 
                 for pixel_position in range(start, start + length)]
    return pixels

# small training set

In [1]:
mask_has_vessel = df_csv['has_vessel']

NameError: name 'df_csv' is not defined

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

AUTOTUNE = tf.data.experimental.AUTOTUNE

import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df_labels = df_csv.groupby('ImageId')['has_vessel'].max()
df_labels = df_labels.astype(str)
df_labels.head(3)

In [ ]:
import pathlib
# data_dir = pathlib.Path('../input/airbus-ship-detection/train_v2
data_dir_raw = pathlib.Path('../input/airbus-ship-detection/train_v2')
data_dir_raw

In [ ]:
mkdir training_small

In [ ]:
mkdir training_small/ship/

In [ ]:
mkdir training_small/no_ship/

In [ ]:
mkdir test_small

In [ ]:
mkdir test_small/ship/

In [ ]:
mkdir test_small/no_ship/

In [ ]:
import shutil

training_size = 50
test_size = 50

counter_ship = 0
counter_no_ship = 0


for item in data_dir_raw.glob('*.jpg'):
    item_str = str(item)
    if df_labels.loc[item.name] == 'True':
        if counter_ship < training_size:
            shutil.copy(item_str, 'training_small/ship/')
            counter_ship += 1
        elif counter_ship < training_size + test_size:
            shutil.copy(item_str, 'test_small/ship/')
            counter_ship += 1
    else:
        if counter_no_ship < training_size:
            shutil.copy(item_str, 'training_small/no_ship/')  
            counter_no_ship += 1
        elif counter_no_ship < training_size + test_size:
            shutil.copy(item_str, 'test_small/no_ship/')
            counter_no_ship += 1
        

In [ ]:
ls training_small/no_ship/ -1 | wc  -l

In [ ]:
ls test_small/ship/ -1 | wc  -l

In [ ]:
ls training_small/ship

In [ ]:
import pathlib
# data_dir = pathlib.Path('../input/airbus-ship-detection/train_v2
data_dir = pathlib.Path('training_small/')
data_dir

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
image_count
# image_count = df_csv_small_training.shape[0]

In [ ]:
BATCH_SIZE = 10
IMG_HEIGHT = 768
IMG_WIDTH = 768
# STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)
epochs = 10

In [ ]:
# CLASS_NAMES = np.array([df_labels.loc[item.name] for item in data_dir.glob('*.jpg')])
# CLASS_NAMES

CLASS_NAMES = np.array([item.name for item in data_dir.glob('*')])
CLASS_NAMES

In [ ]:
ls training_small/

In [ ]:
train = os.listdir('training_small/no_ship')
print(len(train))

In [ ]:
# ls training_small/no_ship

In [ ]:
# The 1./255 is to convert from uint8 to float32 in range [0,1].
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [ ]:
str(data_dir)

In [ ]:
train_data_gen = image_generator.flow_from_directory(directory=str(data_dir), #training_dir
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                      class_mode = 'sparse'
#                                                      classes = list(CLASS_NAMES),
#                                                     color_mode='grayscale',
#                                                     data_format='channels_last'
                                                    )

val_data_gen = validation_image_generator.flow_from_directory(directory='test_small',
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                      class_mode = 'sparse')


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Dropout(0.2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
#     keras.layers.Dense(1, activation = 'sigmoid')
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# model.fit(next(train_data_gen)[0], next(train_data_gen)[1], epochs=20)

In [ ]:
history = model.fit_generator(
    train_data_gen,
#     steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
#     validation_steps=total_val // batch_size
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.predict(val_data_gen)